## Attack on the CDT Gaussian sampling used in FrodoKEM

The process of taking the measurement is described below using two target boards: the STM32F4 and the XMega board.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4' #can be replaced by CWLITEXMEGA if an XMega board is used
VERSION = 'HARDWARE'


In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb" 

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../hardware/victims/firmware/FrodoKem
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2
# Ignore the warnings

In [ ]:
scope.default_setup()

In [ ]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "../hardware/victims/firmware/FrodoKem/GaussianSampling-CW308_STM32F4.hex")
# in case of XMega use  "cw.programmers.XMEGAProgrammer" instead of "cw.programmers.STM32FProgrammer" and change the .hex file accordingly  

In [ ]:
#test that the target board is responsive, it should send back a message 'hello'
target.read()

In [ ]:
# Numbre of traces
N = 50000
# Numbre of samples per trace
scope.adc.samples = 5000

In [ ]:
# Gathering   traces of the execution of the CDT sampler 
 
import matplotlib.pyplot as plt
import os
import numpy as np 


trace = np.zeros((N,scope.adc.samples))
res_dec = np.zeros(N)

for j in range(N):
    scope.arm()
    option = np.random.bytes(2) 
    target.simpleserial_write('p', option) 

    res = target.simpleserial_read('r',2, timeout=1000) 
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    trace[j] = scope.get_last_trace()
    res_dec[j] = twos_comp(res[0],8)

In [ ]:
subtrace = np.zeros((N,750))

for i in range(N):
    #if res_dec[i] >  0 :
    #    res_dec[i]  = res_dec[i]
    #else:
    #    res_dec[i]  = (-1)*res_dec[i]
        
    subtrace[i] = trace[i][0:750]

In [ ]:
# The first 750 samples corresponds to the sampling of one sample by using the CDT sampler
# Difference between postive and negative sampling 


plt.figsize=(6,7)
plt.gcf().set_size_inches(18, 5) 
plt.plot(trace[2][30:800], color = 'blue')  
plt.plot(trace[4][30:800], color = 'red')  
plt.show()
res_dec[0:20]

## Training the Machine-learning Classifiers

In [ ]:
#Similar can be done for Cortex 30 MhZ, and XMega
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(subtrace, res_dec, test_size=0.2)

from sklearn.neural_network import MLPClassifier 

Dec_clf = MLPClassifier(
    random_state=42,
    solver='adam', 
    learning_rate='adaptive',
    max_iter=3000,
        hidden_layer_sizes=[400],
    batch_size=100,
    early_stopping=True,
    n_iter_no_change=100,
    verbose=False, 
)
Dec_clf.fit(X_train, y_train)
 


In [ ]:
predictions =np.zeros(len(y_test))
predictions = Dec_clf.predict(X_test)
success = 0

for i in range(len(y_test)):
    if predictions[i] == y_test[i]: 
        success = success +1
    #else:
        #print(predictions[i], y_test[i])
 
         
print((success /len(y_test))*100)

## Collecting the traces 

### 1- Tracing the sampling of Sp

In [ ]:
#collect the traces of the sampling of S'
#The input of the CDT sampling is stored in the file Sp.txt
from fastapi import File
import matplotlib.pyplot as plt
import os
import numpy as np 

#Security level 1
Sp_size = 10#5120  #PARAMS_N*PARAMS_NBAR = 640 * 8

attack_trace_Sp = np.zeros((Sp_size,scope.adc.samples))
attack_res_dec_Sp = np.zeros(Sp_size)

with open('Sp.txt','r') as f:
    lines = f.readlines()
list_of_values = lines[0].split(' ,')

In [ ]:
for j in range(Sp_size):
    scope.arm()
    option =  int(list_of_values[j]).to_bytes(2, 'little')
    target.simpleserial_write('p', option) 
    res = target.simpleserial_read('r',2, timeout=1000) 
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    attack_trace_Sp[j] = scope.get_last_trace()
    if(twos_comp(res[0],8)<0):
        attack_res_dec_Sp[j] = (-1)*((-1)*twos_comp(res[0],8))>>1
    else:
        attack_res_dec_Sp[j] = twos_comp(res[0]>>1,8)

In [ ]:
# Verify that the sampling is occuring correctly on the target board
# Samples are written in file Spsamples.txt 
plt.figsize=(6,7)
plt.gcf().set_size_inches(18, 5) 
plt.plot(attack_trace_Sp[2][30:800], color = 'blue')  
plt.plot(attack_trace_Sp[4][30:800], color = 'red')  
plt.show()
attack_res_dec_Sp[0:20]


### 2- Tracing the sampling of Epp

In [ ]:
#collect the traces of the sampling of Epp
#The input of the CDT sampling is stored in the file Epp.txt
from fastapi import File
import matplotlib.pyplot as plt
import os
import numpy as np 

#Security level 1
Epp_size = 10 #5120  #PARAMS_N*PARAMS_NBAR = 640 * 8

attack_trace_Epp = np.zeros((Epp_size,scope.adc.samples))
attack_res_dec_Epp = np.zeros(Epp_size)

with open('Epp.txt','r') as f:
    lines = f.readlines()
list_of_values = lines[0].split(' ,')


In [ ]:
for j in range(Epp_size):
    scope.arm()
    option =  int(list_of_values[j]).to_bytes(2, 'little')
    target.simpleserial_write('p', option) 
    res = target.simpleserial_read('r',2, timeout=1000) 
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
    attack_trace_Epp[j] = scope.get_last_trace()
    if(twos_comp(res[0],8)<0):
        attack_res_dec_Epp[j] = (-1)*((-1)*twos_comp(res[0],8))>>1
    else:
        attack_res_dec_Epp[j] = twos_comp(res[0]>>1,8)

In [ ]:
# Verify that the sampling is occuring correctly on the target board
# Samples are written in file Spsamples.txt 
plt.figsize=(6,7)
plt.gcf().set_size_inches(18, 5) 
plt.plot(attack_trace_Epp[2][30:800], color = 'blue')  
plt.plot(attack_trace_Epp[4][30:800], color = 'red')  
plt.show()
 

### Predicting the samples Sp and Epp uying the machine learning classifier

In [ ]:
# predict Sp
Sp_predicted =np.zeros(Sp_size)
Sp_predicted = Dec_clf.predict(attack_trace_Sp)

 

In [ ]:
# predict Epp
Epp_predicted =np.zeros(Sp_size)
Epp_predicted = Dec_clf.predict(attack_trace_Sp)

 

 
